In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.request import Request, urlopen
from urllib.parse import urlparse
from urllib.error import HTTPError, URLError
import requests
import logging
from socket import timeout
import re

In [2]:
politifact_fake = pd.read_csv('fakenewsnet/politifact_fake.csv')
filtered_p_fake = pd.read_csv('data/politifact_fake_with_scraped_text.csv')

In [3]:
politifact_fake

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...
...,...,...,...,...
427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?,954602090462146560\t954602093171609600\t954650...
428,politifact14071,https://web.archive.org/web/20170322070001/htt...,Lindsey Graham Threatens To Convert To Democra...,740351669834244096\t740391312277573632\t740474...
429,politifact13577,http://archive.is/DQqCx,ELECTORAL COLLEGE ELECTOR COMMITS SUICIDE TO A...,810939651900907520\t814738996706164737\t814917...
430,politifact13560,politicono.com/palin-calls-boycott-mall-americ...,Sarah Palin Calls To Boycott Mall Of America B...,805216591424212993\t805222830023540736\t805223...


In [7]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def process_url(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        html = urlopen(req, timeout = 10)
        html = html.read()
        return text_from_html(html)
    except timeout:
        print("connection timedout")
    except URLError as e:
        print(e)
    except HTTPError as err:
        if err.code == 410 or 404:
            print("permanently deleted or removed, url should be removed")
        else:
            raise

In [8]:
# politifact_fake['text'] = ''
for index, url in enumerate(tqdm(politifact_fake['news_url'])):
    if politifact_fake['text'].iloc[index] == '':
        if type(url) != float:
            if urlparse(url).scheme:
                if politifact_fake['text'].iloc[index] == '':
                    politifact_fake['text'].iloc[index] = process_url(url)

  0%|          | 0/432 [00:00<?, ?it/s]

HTTP Error 410: Gone
HTTP Error 404: Not Found
HTTP Error 410: Gone
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 410: Gone
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1124)>
HTTP Error 406: Not Acceptable
<urlopen error [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1124)>
HTTP Error 404: Not Found
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 404: Not Found
<urlopen error [Errno 61] Connection refused>
HTTP Error 410: Gone
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 406: Not Acceptable
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Err

In [9]:
# filtered_p_fake = politifact_fake[politifact_fake['text'] != '' ]
# filtered_p_fake = filtered_p_fake[filtered_p_fake['text'].notnull()]
# filtered_p_fake.to_csv('data/politifact_fake_with_scraped_text.csv')

In [4]:
filtered_p_fake

,Unnamed: 0,id,news_url,title,tweet_ids,text
0,3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,Home Advertis...
1,4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,Washington Sources ...
2,6,politifact13919,http://blogs.trendolizer.com/2015/01/new-york-...,New York Man Wanted For Infecting 240 Men And ...,NaN,Other Trendolizers... Adrenal...
3,7,politifact14795,https://web.archive.org/web/20171027105356/htt...,Saudi Arabia to Behead 6 School Girls for Bein...,923126512458616832\t923135295070990341\t923189...,success fail ...
4,8,politifact14328,https://web.archive.org/web/20170702174006/htt...,Malia Obama Fired From Cushy Internship At Spa...,880455776107679747\t880457763876462598\t880461...,success fail ...
...,...,...,...,...,...,...
228,419,politifact14169,https://web.archive.org/web/20170528095037/htt...,Rubio: “Rape Victims Should Be In Custody If T...,663538460104392704\t663757208031780864\t663764...,success fail ...
229,421,politifact14992,http://www.trainnews.info/2018/01/rep-paul-gos...,Rep. Paul Gosar Asks Capitol Police to Arrest ...,958425300144218112\t958428242528145409\t958428...,Home Business Online Business Bitcoin Revi...
230,422,politifact14158,https://web.archive.org/web/20170602190500/htt...,WORSE THAN HITLER! Trey Gowdy’s Son Found In A...,865933040492703745,success fail ...
231,427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?,954602090462146560\t954602093171609600\t954650...,...


In [3]:
from gensim.corpora import Dictionary

MIN_DF = 1 # minium document frequency
MAX_DF = 0.6 # maximum document frequency

In [4]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')


In [5]:
tokenized_texts = []
for index, text in tqdm(enumerate(filtered_p_fake['text'])):
    tokenized_texts.append(tokenizer.tokenize(text.lower()))
        
# tokenized_text = tokenizer.tokenize(sample_text.lower())
# print(tokenized_text)

In [6]:
dictionary = Dictionary(tokenized_texts) # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF, 
                           no_above=MAX_DF)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [9]:
import os
os.environ['MALLET_HOME'] = 'mallet-2.0.8'

In [18]:
from gensim.models.wrappers import LdaMallet

PATH_TO_MALLET = 'mallet-2.0.8/bin/mallet'
N_TOPICS = 10
N_ITERATIONS = 500

lda = LdaMallet(mallet_path=PATH_TO_MALLET,
                corpus=corpus,
                id2word=dictionary,
                num_topics=N_TOPICS,
                iterations=N_ITERATIONS)

In [22]:
for topic in range(N_TOPICS):
    words = lda.show_topic(topic, 10)
    topic_n_words = ' '.join([word[0] for word in words])
    print('Topic {}: {}'.format(str(topic), topic_n_words))

Topic 0: share click opens window twitter facebook 0 presidential google shares
Topic 1: 2017 reply 2 march pm 3 domain 1 don whoopi
Topic 2: federal public search government tweet information report 04 opinion law
Topic 3: trump president donald comments election log house world white obama
Topic 4: 2021 biden april comments 3 facebook 4 joe 30 politics
Topic 5: stats trending trendolizer video contact general trump sports site twitter
Topic 6: email source morning show video 70 smiley rickey contact advertisement
Topic 7: 2017 trump archive breaking internet web machine wayback data day
Topic 8: 2020 2021 2018 2019 february 2016 world april november december
Topic 9: trump politics entertainment health facebook policy popular world video 2021
